## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from  config import g_key
import json
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("df")
city_data_df.head() 
city_data_df =city_data_df.rename(columns={"County":"Country"}) 
city_data_df.drop(columns="Unnamed: 0")

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,mulchen,cl,-37.7167,-72.2333,55.47,40,2,7.851659,clear sky
1,yellowknife,ca,62.4560,-114.3525,19.42,92,100,11.497872,light snow
2,krasnoselkup,ru,65.7000,82.4667,27.88,95,99,12.683450,overcast clouds
3,tuatapere,nz,-46.1333,167.6833,66.36,62,1,1.655336,clear sky
4,thompson,ca,55.7435,-97.8558,28.56,54,75,10.357032,broken clouds
...,...,...,...,...,...,...,...,...,...
1995,coquimbo,cl,-29.9533,-71.3436,62.33,72,0,9.216193,clear sky
1996,grand gaube,mu,-20.0064,57.6608,76.37,76,14,2.997500,few clouds
1997,avarua,ck,-21.2078,-159.7750,84.25,79,40,11.497872,scattered clouds
1998,barrow,us,71.2906,-156.7887,-9.38,70,0,24.158952,clear sky


In [3]:
# 2a. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("Please enter a minimum temperature in Farenheight")) 

Please enter a minimum temperature in Farenheight70


In [4]:
# 2b. Prompt the user to enter maximum temperature criteria
max_temp = float(input("Please enter a maximum temperature in Farenheight"))

Please enter a maximum temperature in Farenheight90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
8,8,ambon,id,-3.6954,128.1814,80.74,83,20,2.304048,few clouds
9,9,port hedland,au,-20.3167,118.5667,82.45,69,17,12.661080,few clouds
12,12,butaritari,ki,3.0707,172.7902,81.07,69,100,11.967629,overcast clouds
14,14,contamana,pe,-7.3333,-75.0167,80.10,67,79,3.221194,broken clouds
17,17,coromandel,nz,-36.7667,175.5000,74.37,56,9,4.898899,clear sky
19,19,emerald,au,-23.5333,148.1667,76.33,56,0,15.233561,clear sky
20,20,souillac,mu,-20.5167,57.5167,75.58,83,40,0.000000,scattered clouds
22,22,carnarvon,au,-24.8667,113.6333,71.67,94,49,6.912145,scattered clouds
25,25,cayenne,gf,4.9333,-52.3333,80.10,1,0,5.748936,clear sky
26,26,bengkulu,id,-3.8004,102.2655,75.18,85,90,5.793675,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
city_data_df.isna().any()

Unnamed: 0             False
City                   False
Country                False
Lat                    False
Lng                    False
Max Temp               False
Humidity               False
Cloudiness             False
Wind Speed             False
Current Description    False
dtype: bool

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
city_data_df = city_data_df.dropna()
city_data_df

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,mulchen,cl,-37.7167,-72.2333,55.47,40,2,7.851659,clear sky
1,1,yellowknife,ca,62.4560,-114.3525,19.42,92,100,11.497872,light snow
2,2,krasnoselkup,ru,65.7000,82.4667,27.88,95,99,12.683450,overcast clouds
3,3,tuatapere,nz,-46.1333,167.6833,66.36,62,1,1.655336,clear sky
4,4,thompson,ca,55.7435,-97.8558,28.56,54,75,10.357032,broken clouds
...,...,...,...,...,...,...,...,...,...,...
1995,1995,coquimbo,cl,-29.9533,-71.3436,62.33,72,0,9.216193,clear sky
1996,1996,grand gaube,mu,-20.0064,57.6608,76.37,76,14,2.997500,few clouds
1997,1997,avarua,ck,-21.2078,-159.7750,84.25,79,40,11.497872,scattered clouds
1998,1998,barrow,us,71.2906,-156.7887,-9.38,70,0,24.158952,clear sky


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = city_data_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,mulchen,cl,55.47,clear sky,-37.7167,-72.2333,
1,yellowknife,ca,19.42,light snow,62.4560,-114.3525,
2,krasnoselkup,ru,27.88,overcast clouds,65.7000,82.4667,
3,tuatapere,nz,66.36,clear sky,-46.1333,167.6833,
4,thompson,ca,28.56,broken clouds,55.7435,-97.8558,
5,sakleshpur,in,68.54,broken clouds,12.9667,75.7833,
6,chifeng,cn,28.74,broken clouds,42.2683,118.9636,
7,brae,gb,35.65,overcast clouds,60.3964,-1.3530,
8,ambon,id,80.74,few clouds,-3.6954,128.1814,
9,port hedland,au,82.45,few clouds,-20.3167,118.5667,


In [30]:
# 6a. Set parameters to search for hotels with 5000 meters.

params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,   
    "location": None
}

base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"  
hotels = requests.get(base_url, params=params).json()  
for row in hotel_df.itertuples(index=True):
    lat=row.Lat 
    long= row.Lng 
    base_url 
    params["location"]="{},{}".format(lat,long) 
    hotels = requests.get(base_url, params=params).json()
    if hotels["status"] != "OK":
        continue
    if not hotels["results"]:
        continue
    result = hotels["results"][0]
    name = result["name"]
    hotel_df.loc[row.Index, "Hotel Name"]=name
      
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,mulchen,cl,55.47,clear sky,-37.7167,-72.2333,Hotel Los Maitenes de Villalón
1,yellowknife,ca,19.42,light snow,62.4560,-114.3525,The Explorer Hotel
2,krasnoselkup,ru,27.88,overcast clouds,65.7000,82.4667,Krasnosel'kupnefteprodukt
3,tuatapere,nz,66.36,clear sky,-46.1333,167.6833,Ron and Tony's Bed & Breakfast
4,thompson,ca,28.56,broken clouds,55.7435,-97.8558,Burntwood Hotel
...,...,...,...,...,...,...,...
1995,coquimbo,cl,62.33,clear sky,-29.9533,-71.3436,Hotel San Juan
1996,grand gaube,mu,76.37,few clouds,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
1997,avarua,ck,84.25,scattered clouds,-21.2078,-159.7750,Paradise Inn
1998,barrow,us,-9.38,clear sky,71.2906,-156.7887,King Eider Inn


In [34]:
# 7. Drop the rows where there is no Hotel Name. 
clean_hotel_df=hotel_df[hotel_df["Hotel Name"] != ""] 
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,mulchen,cl,55.47,clear sky,-37.7167,-72.2333,Hotel Los Maitenes de Villalón
1,yellowknife,ca,19.42,light snow,62.4560,-114.3525,The Explorer Hotel
2,krasnoselkup,ru,27.88,overcast clouds,65.7000,82.4667,Krasnosel'kupnefteprodukt
3,tuatapere,nz,66.36,clear sky,-46.1333,167.6833,Ron and Tony's Bed & Breakfast
4,thompson,ca,28.56,broken clouds,55.7435,-97.8558,Burntwood Hotel
...,...,...,...,...,...,...,...
1995,coquimbo,cl,62.33,clear sky,-29.9533,-71.3436,Hotel San Juan
1996,grand gaube,mu,76.37,few clouds,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
1997,avarua,ck,84.25,scattered clouds,-21.2078,-159.7750,Paradise Inn
1998,barrow,us,-9.38,clear sky,71.2906,-156.7887,King Eider Inn


In [35]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [52]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [55]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))